# CCDC Submission

### LIBRARIES

In [1]:
import shutil
import rasterio
import pyproj
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas_path import path  # noqa
from pathlib import Path
from PIL import Image
import pytorch_lightning as pl
import torch

In [4]:
from train_src.cloud_model import CloudModel

### LOAD MODEL WEIGHTS

| No | Model | Backbone | Weights | Folder | Accuracy |
| --- | --- | --- | -- | -- | -- | 
| 1 | unet | resnet50 | imagenet | lightning_logs/version_9/checkpoints/epoch=33-step=33455.ckpt | .883 |

In [5]:
# Baseline submission.
model = CloudModel.load_from_checkpoint(checkpoint_path="lightning_logs/version_9/checkpoints/epoch=33-step=33455.ckpt")

### GENERATE A SUBMISSION

In [6]:
# create benchmark_src folder
submission_dir = Path("submission")

In [5]:
# save the model
submission_assets_dir = submission_dir / "assets"
submission_assets_dir.mkdir(parents=True, exist_ok=True)

model_weight_path = submission_assets_dir / "cloud_model.pt"
torch.save(model.state_dict(), model_weight_path)

#### main.py

In [9]:
%%file {submission_dir}/main.py
import os
from pathlib import Path
from typing import List

from loguru import logger
import pandas as pd
from PIL import Image
import torch
import typer

try:
    from cloud_dataset import CloudDataset
    from cloud_model import CloudModel
except ImportError:
    from submission.cloud_dataset import CloudDataset
    from submission.cloud_model import CloudModel


ROOT_DIRECTORY = Path("/codeexecution")
PREDICTIONS_DIRECTORY = ROOT_DIRECTORY / "predictions"
ASSETS_DIRECTORY = ROOT_DIRECTORY / "assets"
DATA_DIRECTORY = ROOT_DIRECTORY / "data"
INPUT_IMAGES_DIRECTORY = DATA_DIRECTORY / "test_features"

# Set the pytorch cache directory and include cached models in your submission.zip
os.environ["TORCH_HOME"] = str(ASSETS_DIRECTORY / "assets/torch")


def get_metadata(features_dir: os.PathLike, bands: List[str]):
    """
    Given a folder of feature data, return a dataframe where the index is the chip id
    and there is a column for the path to each band's TIF image.

    Args:
        features_dir (os.PathLike): path to the directory of feature data, which should have
            a folder for each chip
        bands (list[str]): list of bands provided for each chip
    """
    chip_metadata = pd.DataFrame(index=[f"{band}_path" for band in bands])
    chip_ids = (
        pth.name for pth in features_dir.iterdir() if not pth.name.startswith(".")
    )

    for chip_id in chip_ids:
        chip_bands = [features_dir / chip_id / f"{band}.tif" for band in bands]
        chip_metadata[chip_id] = chip_bands

    return chip_metadata.transpose().reset_index().rename(columns={"index": "chip_id"})


def make_predictions(
    model: CloudModel,
    x_paths: pd.DataFrame,
    bands: List[str],
    predictions_dir: os.PathLike,
):
    """Predicts cloud cover and saves results to the predictions directory.

    Args:
        model (CloudModel): an instantiated CloudModel based on pl.LightningModule
        x_paths (pd.DataFrame): a dataframe with a row for each chip. There must be a column for chip_id,
                and a column with the path to the TIF for each of bands provided
        bands (list[str]): list of bands provided for each chip
        predictions_dir (os.PathLike): Destination directory to save the predicted TIF masks
    """
    test_dataset = CloudDataset(x_paths=x_paths, bands=bands)
    test_dataloader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=model.batch_size,
        num_workers=model.num_workers,
        shuffle=False,
        pin_memory=True,
    )

    for batch_index, batch in enumerate(test_dataloader):
        logger.debug(f"Predicting batch {batch_index} of {len(test_dataloader)}")
        x = batch["chip"]
        preds = model.forward(x)
        preds = torch.softmax(preds, dim=1)[:, 1]
        preds = (preds > 0.5).detach().numpy().astype("uint8")
        for chip_id, pred in zip(batch["chip_id"], preds):
            chip_pred_path = predictions_dir / f"{chip_id}.tif"
            chip_pred_im = Image.fromarray(pred)
            chip_pred_im.save(chip_pred_path)


def main(
    model_weights_path: Path = ASSETS_DIRECTORY / "cloud_model.pt",
    test_features_dir: Path = DATA_DIRECTORY / "test_features",
    predictions_dir: Path = PREDICTIONS_DIRECTORY,
    bands: List[str] = ["B02", "B03", "B04", "B08"],
    fast_dev_run: bool = False,
):
    """
    Generate predictions for the chips in test_features_dir using the model saved at
    model_weights_path.

    Predictions are saved in predictions_dir. The default paths to all three files are based on
    the structure of the code execution runtime.

    Args:
        model_weights_path (os.PathLike): Path to the weights of a trained CloudModel.
        test_features_dir (os.PathLike, optional): Path to the features for the test data. Defaults
            to 'data/test_features' in the same directory as main.py
        predictions_dir (os.PathLike, optional): Destination directory to save the predicted TIF masks
            Defaults to 'predictions' in the same directory as main.py
        bands (List[str], optional): List of bands provided for each chip
    """
    if not test_features_dir.exists():
        raise ValueError(
            f"The directory for test feature images must exist and {test_features_dir} does not exist"
        )
    predictions_dir.mkdir(exist_ok=True, parents=True)

    logger.info("Loading model")
    model = CloudModel(bands=bands, hparams={"weights": None})
    model.load_state_dict(torch.load(model_weights_path))

    logger.info("Loading test metadata")
    test_metadata = get_metadata(test_features_dir, bands=bands)
    if fast_dev_run:
        test_metadata = test_metadata.head()
    logger.info(f"Found {len(test_metadata)} chips")

    logger.info("Generating predictions in batches")
    make_predictions(model, test_metadata, bands, predictions_dir)

    logger.info(f"""Saved {len(list(predictions_dir.glob("*.tif")))} predictions""")


if __name__ == "__main__":
    typer.run(main)

Overwriting submission/main.py


### Test

In [11]:
DATA_DIR = Path("/driven-data/cloud-cover")
TRAIN_FEATURES = DATA_DIR / "train_features"

In [12]:
from submission.main import main

main(
    model_weights_path=submission_dir / "assets/cloud_model.pt",
    test_features_dir=TRAIN_FEATURES,
    predictions_dir=submission_dir / "predictions",
    fast_dev_run=True,
)

2022-01-23 09:51:55.105 | INFO     | submission.main:main:115 - Loading model
2022-01-23 09:51:55.619 | INFO     | submission.main:main:119 - Loading test metadata
2022-01-23 09:52:04.067 | INFO     | submission.main:main:123 - Found 5 chips
2022-01-23 09:52:04.068 | INFO     | submission.main:main:125 - Generating predictions in batches
2022-01-23 09:52:06.335 | DEBUG    | submission.main:make_predictions:76 - Predicting batch 0 of 1
2022-01-23 09:52:13.138 | INFO     | submission.main:main:128 - Saved 5 predictions


In [10]:
# Remove predictions folder
!rm -rf {submission_dir}/predictions

### ZIP SUMBSISISSION FILES

In [11]:
# clear out our pycache before zipping up submission
!rm -rf {submission_dir}/__pycache__

In [12]:
!tree {submission_dir}

submission
├── assets
│   └── cloud_model.pt
├── cloud_dataset.py
├── cloud_model.py
├── losses.py
└── main.py

1 directory, 5 files


In [15]:
# Zip submission
!cd {submission_dir} && zip -r ../submission.zip *

  adding: assets/ (stored 0%)
  adding: assets/cloud_model.pt (deflated 7%)
  adding: cloud_dataset.py (deflated 63%)
  adding: cloud_model.py (deflated 74%)
  adding: losses.py (deflated 57%)
  adding: main.py (deflated 64%)


In [16]:
!du -h submission.zip

116M	submission.zip
